In [1]:
import torch

In [2]:
import pandas as pd

In [6]:
pwd

'C:\\Users\\jeong\\proj\\nlp_paper\\py'

In [10]:
dv = pd.read_pickle('../data/oil_news_dvec.pkl')

In [9]:
df = pd.read_pickle('../data/oil_news_df.pkl')

In [11]:
df.head()

,mon,keyword,rescnt,title,date,wkday,text,smry,press
0,201301,oil price,,Iran's Real Threat to the West,,,Since the Islamic revolution in Iran in 1979 m...,The real danger emanating from Iran today come...,oilprice.com
1,201301,oil price,,Subscribe to read,,,Become an FT subscriber to read:\n\nLeverage o...,Become an FT subscriber to read:\nLeverage our...,www.ft.com
2,201301,oil price,,A Graphic View of How Shale Gas could Change t...,,,The real impact stems from its effect on the o...,...The major geopolitical impact of shale extr...,oilprice.com
3,201301,oil price,,Government allows oil firms to revise diesel p...,,,In a move that could help the government reduc...,It had proposed a Rs 3-4.50 per litre hike in ...,www.ndtv.com
4,201301,oil price,,Trillions of dollars worth of oil found in Aus...,,,The discovery in central Australia was reporte...,"“What we’re seeing up there is a very, very bi...",www.telegraph.co.uk


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [13]:
torch.manual_seed(1)

In [14]:
word_to_ix = {"hello":0,"world":1}

In [15]:
embeds = nn.Embedding(2,5) # 2 words in vocab, 5 dimensional mbeddings

In [16]:
lookup_tensor = torch.tensor([word_to_ix["hello"]],dtype = torch.long)
hello_embed = embeds(lookup_tensor)

In [17]:
hello_embed

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)

In [20]:
[word_to_ix["hello"]]

[0]

In [21]:
torch.tensor([0],dtype=torch.long)

tensor([0])

In [22]:
lookup_tensor

tensor([0])

In [24]:
embeds(lookup_tensor)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward>)

In [25]:
context_size = 2
embedding_dim = 10

In [26]:
df.head()

,mon,keyword,rescnt,title,date,wkday,text,smry,press
0,201301,oil price,,Iran's Real Threat to the West,,,Since the Islamic revolution in Iran in 1979 m...,The real danger emanating from Iran today come...,oilprice.com
1,201301,oil price,,Subscribe to read,,,Become an FT subscriber to read:\n\nLeverage o...,Become an FT subscriber to read:\nLeverage our...,www.ft.com
2,201301,oil price,,A Graphic View of How Shale Gas could Change t...,,,The real impact stems from its effect on the o...,...The major geopolitical impact of shale extr...,oilprice.com
3,201301,oil price,,Government allows oil firms to revise diesel p...,,,In a move that could help the government reduc...,It had proposed a Rs 3-4.50 per litre hike in ...,www.ndtv.com
4,201301,oil price,,Trillions of dollars worth of oil found in Aus...,,,The discovery in central Australia was reporte...,"“What we’re seeing up there is a very, very bi...",www.telegraph.co.uk


In [37]:
from nltk.tokenize import word_tokenize

In [38]:
test_sentence = word_tokenize(df.text[0])

In [39]:
trigrams = [([test_sentence[i],test_sentence[i+1]],test_sentence[i+2]) for i in range(len(test_sentence)-2)]

In [40]:
print(trigrams[:3])

[(['Since', 'the'], 'Islamic'), (['the', 'Islamic'], 'revolution'), (['Islamic', 'revolution'], 'in')]


In [41]:
vocab = set(test_sentence)

In [42]:
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [43]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

NameError: name 'EMBEDDING_DIM' is not defined